<a href="https://colab.research.google.com/github/hbisgin/BigDatav1/blob/main/Lecture21_MNIST_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ---------------------------
# 1. Load MNIST
# ---------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=1000, shuffle=False)

# ---------------------------
# 2. Define 1-hidden-layer MLP
# ---------------------------
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(28*28, 10)   # hidden layer
        self.output = nn.Linear(10, 10)      # output layer

    def forward(self, x):
        x = x.view(x.size(0), -1)             # flatten input
        h = torch.relu(self.hidden(x))
        o = self.output(h)                    # raw logits
        return o

model = MLP()

# ---------------------------
# 3. Loss & Optimizer
# ---------------------------
criterion = nn.CrossEntropyLoss()  # includes softmax internally
optimizer = optim.SGD(model.parameters(), lr=0.01)

# ---------------------------
# 4. Training Loop
# ---------------------------
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for X, y in train_loader:
        optimizer.zero_grad()

        logits = model(X)
        loss = criterion(logits, y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.3f}")

# ---------------------------
# 5. Evaluation
# ---------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X, y in test_loader:
        logits = model(X)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"Test Accuracy: {correct/total:.4f}")


Epoch 1/5, Loss: 686.432
Epoch 2/5, Loss: 335.167
Epoch 3/5, Loss: 294.952
Epoch 4/5, Loss: 274.906
Epoch 5/5, Loss: 262.082
Test Accuracy: 0.9207
